In [ ]:
using JLD, Mamba
using Pipe
include("gyncycle.jl")

acceptance(mc::Array) = mapslices(mc,[1,2]) do mc
    n = size(mc,1)
    sum([mc[i,:,:] != mc[i+1,:,:] for i in 1:n-1]) / (n-1);
  end |> vec

acceptance(c::AbstractChains) = acceptance(c.value)

mergeparms!(aparms, sparms) = (aparms[SAMPLEPARMS]=sparms; aparms)

mergeparms(aparms,sparms) = mergeparms!(copy(aparms), sparms)

function loglikelihood(mc::Array{Float64,3}, data::Matrix)
    mapslices(mc.value, 2) do modelparms
        aparms = mergeparms(parms, modelparms[1:length(SAMPLEPARMS)])
        y0 = modelparms[length(SAMPLEPARMS)+1:end]
        loglikelihood(data, aparms, y0)
    end
end

mergepersons(mcs) = cat(3,map(x->x[1:minimum(map(x->size(x,1), mcs)),:,:],mcs)...);

loglikelihood(mc::AbstractChains, data::Matrix) = loglikelihood(mc.value, data);

function newestjld(dir)
    jlds = filter(x->endswith(x, "jld"), readdir(dir))
    file = jlds[indmax(map(ctime, jlds))]
    joinpath(dir, file)
end;

In [ ]:
file = newestjld("out")
@time mcs = load(file)["mc"] |> mergepersons
mcs = Chains(mcs, names = vcat(parameternames[SAMPLEPARMS], speciesnames))
println("file: ", file)
println("size: ", size(mcs))
println("acc:  ", map(x->round(x,2),acceptance(mcs)))
draw(plot(mcs[:,[1:10,83:92],:]), ask=false, fmt=:png)